In [12]:
import sys
sys.path.insert(1, '/home/jupyter/crisp')
import os
import pandas as pd
from tqdm import tqdm

In [3]:
from models.OpenFLTorchModelZoo import *

In [8]:
# packages imported in the tutorial
import pandas as pd
import pickle
from synthetic.synthetic_generator import synthetic_generator


import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet

from synthetic.facebook_synthetic_data_generator import generator_example

In [7]:
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── save
├── data
├── cert
├── .workspace
├── plan
│   ├── data.yaml
│   ├── cols.yaml
│   ├── defaults
│   │   ├── network.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── task_runner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── compression_pipeline.yaml
│   │   ├── collaborator.yaml
│   │   ├── tasks_keras.yaml
│   │   └── aggregator.yaml
│   └── plan.yaml
├── logs
├── code
│   ├── ptmnist_inmemory.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── pt_cnn.py
└── requirements.txt

7 directories, 21 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing 

## Data 

In [14]:
# Data Upload from the FB set
data_path = 'data/synthetic'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
n_example = 1
dim_inv=2
dim_spu=2
n_exp=int(1e3)
n_env=2 
save_dir= data_path
test=False 

generator_example(n_example, dim_inv, dim_spu, n_exp, n_env ,save_dir, test)

data_name = "data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test)
df_train = pd.read_pickle(os.path.join(data_path, data_name))


Environments variables: {'E0': 0.1, 'E1': 1.5}
Generated Synthetic Data according to the Facebook setup Example: 1


In [20]:
df_train.keys()

Index(['Causal_0', 'Causal_1', 'Non_causal_0', 'Non_causal_1', 'env_split',
       'Subj_ID', 'Target'],
      dtype='object')

In [54]:
y = df_train['Target'].to_numpy()
df_train.drop(columns=['Subj_ID','Target'])
X = df_train.to_numpy()

In [132]:
# Data division into training and validation
thre = int(0.7*len(y))
X_train = torch.from_numpy(X[:thre,:]).float()
y_train = torch.from_numpy(X[:thre,:]@np.ones((X.shape[1],))).float().unsqueeze(1)
X_test = torch.from_numpy(X[thre:,:]).float()
y_test = torch.from_numpy(X[thre:,:]@np.ones((X.shape[1],))).float().unsqueeze(1)

In [140]:
# np.expand_dims(train_images, axis=1)
fl_data = FederatedDataSet(X_train,y_train,X_test,y_test,batch_size=32,
                           num_classes=1) 
### Not sure how to use classes when we are actually doing a regression
# Not sure it's actually necessary since we are choosing the loss function
# Update, it's necessary as they try to implement argmax on the output and it returns an error

## Model

In [134]:
class TorchLinearRegressionModule(torch.nn.Module):
    def __init__(self, inputSize=7, outputSize=1, bias=True):
        super(TorchLinearRegressionModule, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize, bias=bias)

    def forward(self, x):
        out = self.linear(x)
        return out.unsqueeze(0)

In [227]:
df = synthetic_generator()
df["Target"] = np.random.randint(0,4,size=df.shape[0])
data = torch.tensor(df[df.columns[:350]].values).float()
targets = torch.tensor(df[df.columns[350]].values)

In [229]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df[df.columns[:350]].values, train_size = 0.8)
y_train, y_test = train_test_split(df[df.columns[350]].values, train_size = 0.8)
X_train = torch.tensor(X_train).float()
X_test = torch.tensor(X_test).float()
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [230]:
fl_data = FederatedDataSet(X_train,y_train,X_test,y_test,batch_size=32,
                           num_classes=4) 

In [231]:
class OpenFLTorchLogisticRegressionModule(torch.nn.Module):
    def __init__(self, 
                 inputSize=data.shape[1], 
                 outputSize=len(targets.unique()), 
                 bias=True, 
                 ):
        
        super(OpenFLTorchLogisticRegressionModule, self).__init__()
        self.outputSize = outputSize
        self.linear = torch.nn.Linear(inputSize, self.outputSize, bias=bias)
        
    def forward(self, x):
        output = self.linear(x)
        return output
#       return (output, self.coef_)
#    def coef_(self):
#       return self.linear.weight.detach()

In [253]:
optimizer = lambda x: torch.optim.SGD(x, lr=0.001, weight_decay = 0.5) 
def loss(output, target):
    return torch.nn.functional.cross_entropy(output, target)

In [265]:
model = OpenFLTorchLogisticRegressionModule()
epochs = 50
model_optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay = 0.5)

In [266]:
for epoch in tqdm(range(epochs)):
    #forward feed
    y_pred = model.forward(X_train.requires_grad_())

    criteria = loss(y_pred, y_train)
    print
    #backward propagation: calculate gradients
    criteria.backward()

    #update the weights
    model_optimizer.step()

    #clear out the gradients from the last step loss.backward()
    model_optimizer.zero_grad()

    
# def loss(output, target, l1Coef = 0.5):
#     torch.nn.MSELoss(output[0] - target) + (l1Coef * torch.norm(output[1],1))

100%|██████████| 50/50 [00:00<00:00, 2120.03it/s]


In [267]:
criteria

tensor(1.2530, grad_fn=<NllLossBackward>)

In [258]:
#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=OpenFLTorchLogisticRegressionModule,
                          optimizer=optimizer,loss_fn=loss,
                          data_loader=fl_data)

# Training

In [259]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [260]:
final_fl_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':5})

/opt/conda/lib/python3.7/site-packages/openfl/federated/task/runner_pt.py:101: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(
/opt/conda/lib/python3.7/site-packages/openfl/federated/task/runner_pt.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target).to(self.device, dtype=pt.int64)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [261]:
model

OpenFLTorchLogisticRegressionModule(
  (linear): Linear(in_features=350, out_features=4, bias=True)
)

# Testing the CLI

In [1]:
import sys
sys.path.insert(1, '/home/jupyter/crisp')
import os
import pandas as pd
from tqdm import tqdm

In [5]:
import pandas as pd
import pickle
from synthetic.synthetic_generator import synthetic_generator


import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet

from synthetic.facebook_synthetic_data_generator import generator_example

In [6]:
fx workspace create --prefix ~/CRISP_workspace --template torch_cnn_mnist

SyntaxError: invalid syntax (<ipython-input-6-aac69ddf923c>, line 1)

In [7]:
fx aggregator start

SyntaxError: invalid syntax (<ipython-input-7-7ae226b183a8>, line 1)